<a href="https://colab.research.google.com/github/Chai567/TigerSupplyChainStream/blob/main/SupplyChainAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Supply Chain Analytics**


---
Supply chain analytics refers to the processes organizations use to gain insight and extract value from the large amounts of data associated with the procurement, processing and distribution of goods.




I have used Supply Chain starter kit and analyzed data returned by various query with interactive UI using tools such as Graphistry, plotly, etc.

Install the required packages such as plotly, pyTigerGraph, graphistry, etc.

In [ ]:
!pip install plotly --upgrade

In [ ]:
!pip install pyTigerGraph

In [ ]:
!pip install --user graphistry

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
import pyTigerGraph as tg 

# Connect to the TigerGraph solution
#Here, the supply chain graph name is demo_graph
conn = tg.TigerGraphConnection(host="https://supply.i.tgcloud.io/", password="tigergraph", graphname="demo_graph")
conn.apiToken = conn.getToken(conn.createSecret())

In [ ]:
#set up graph connection
conn.gsql('ls')

'---- Graph demo_graph\nVertex Types:\n- VERTEX product(PRIMARY_ID pid STRING, name STRING, price FLOAT, formula STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"\n- VERTEX site(PRIMARY_ID sid STRING, name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"\n- VERTEX p_order(PRIMARY_ID orderId STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"\n- VERTEX stocking(PRIMARY_ID stockingId STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"\nEdge Types:\n- DIRECTED EDGE usedBy(FROM product, TO product, formula_order STRING, useAmount FLOAT) WITH REVERSE_EDGE="reverseUsedBy"\n- DIRECTED EDGE reverseUsedBy(FROM product, TO product, formula_order STRING, useAmount FLOAT) WITH REVERSE_EDGE="usedBy"\n- DIRECTED EDGE deliver(FROM site, TO site, itemId STRING) WITH REVERSE_EDGE="reverseDeliver"\n- DIRECTED EDGE reverseDeliver(FROM site, TO site, itemId STRING) WITH REVERSE_EDGE="deliver"\n- DIRECTED E

In [ ]:
#connect to graphistry
import graphistry
#graphistry connection details
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="Chaitanya", password="Abcd@1234")

In [ ]:
import pandas as pd 
import numpy as np
import plotly.express as px

**showWholeGraph**
This query will return all the edges and vertices of the graph


In [ ]:
#running showWholeGraph query, processing and analysing results using graphistry
showGraph = conn.runInstalledQuery("showWholeGraph") 
print(showGraph)

[{'Start': [{'v_id': 'site14', 'v_type': 'site', 'attributes': {'name': 'site14'}}, {'v_id': 'ALUMINUM RODS1467411917', 'v_type': 'stocking', 'attributes': {}}, {'v_id': 'CAR WHEEL SET1492012930', 'v_type': 'p_order', 'attributes': {}}, {'v_id': 'CAR RIMS', 'v_type': 'product', 'attributes': {'name': 'CAR RIMS', 'price': 80, 'formula': '$1+50+$2'}}, {'v_id': 'VALVE STEM1467411893', 'v_type': 'stocking', 'attributes': {}}, {'v_id': 'RUBBER1467411803', 'v_type': 'stocking', 'attributes': {}}, {'v_id': 'TIRES', 'v_type': 'product', 'attributes': {'name': 'TIRES', 'price': 100, 'formula': '($1+$2)*0.8'}}, {'v_id': 'site3', 'v_type': 'site', 'attributes': {'name': 'site3'}}, {'v_id': 'STEEL RODS1467411848', 'v_type': 'stocking', 'attributes': {}}, {'v_id': 'RUBBER', 'v_type': 'product', 'attributes': {'name': 'RUBBER', 'price': 10, 'formula': ''}}, {'v_id': 'CAR WHEEL SET', 'v_type': 'product', 'attributes': {'name': 'CAR WHEEL SET', 'price': 840, 'formula': '$1+10+$2*0.5'}}, {'v_id': 'ALUM

In [ ]:
graph_df = pd.DataFrame(showGraph[1]['@@displaySet'])
print(graph_df)

          e_type               from_id from_type         to_id  to_type  \
0        deliver                 site8      site        site14     site   
1        produce                site11      site        RUBBER  product   
2        deliver                 site5      site         site4     site   
3         usedBy                 BELTS   product         TIRES  product   
4   prodStocking  CONTROLLER1467411945  stocking    CONTROLLER  product   
..           ...                   ...       ...           ...      ...   
56        usedBy            STEEL RODS   product         BELTS  product   
57       produce                 site5      site    STEEL RODS  product   
58        usedBy                SENSOR   product  TPM ASSEMBLY  product   
59       produce                site14      site    ABS MODULE  product   
60       deliver                site14      site         site4     site   

    directed                                attributes  
0       True                  {'itemId': '

In [ ]:
supply_chain_graph = graphistry.edges(graph_df, 'from_id', 'to_id')

In [ ]:
#plotting the entire supply chain graph
supply_chain_graph.plot()

Failed memoization speedup attempt due to Pandas internal hash function failing. Continuing without memoization speedups.This is fine, but for speedups around skipping re-uploads of previously seen tables, try identifying which columns have types that Pandas cannot hash, and convert them to hashable types like strings.


**checkStocking**
This query calculate the amount of raw material needed to produce the required quantity of product and if the current production capacity is capable of fulfilling the requested demand.

In [ ]:
import ipywidgets as widgets
site_list=['site1','site2','site3','site4','site5','site7','site8','site9','site10','site11','site12','site13','site14','site15']
product_list=['CAR WHEEL SET','TRUCK WHEEL SET','TIRES','TRUCK RIMS','CAR RIMS']
material_list=['RUBBER','TIRES','STEEL RODS','ALUMINUM RODS','SENSOR','BELTS']
price_increment_list=[0.2,0.4,0.6,0.8,1]
ordering_amount_list=[2,4,5,8,10]
max_iteration_list=[2,4,5,8,10]
max_depth_list=[2,4,5,8,10]

In [58]:
#select product name and product quantity 
product_drop_down=widgets.Dropdown(
    options=product_list,
    description='Select a product name',
    disabled=False,
)
display(product_drop_down)
order_amount_drop_down=widgets.Dropdown(
    options=ordering_amount_list,
    description='Select the amount of product ordered ',
    disabled=False,
)
display(order_amount_drop_down)

Dropdown(description='Select a product name', options=('CAR WHEEL SET', 'TRUCK WHEEL SET', 'TIRES', 'TRUCK RIM…

Dropdown(description='Select the amount of product ordered ', options=(2, 4, 5, 8, 10), value=2)

In [ ]:
#run checkStocking query
product=product_drop_down.value
order_amount=order_amount_drop_down.value
params = {"input":product, "amount":order_amount} #query's arguments
queryName = "checkStocking"
checkStock = conn.runInstalledQuery(queryName, params) 
print(checkStock)

[{'Start': [{'v_id': 'TRUCK WHEEL SET', 'v_type': 'product', 'attributes': {'name': 'TRUCK WHEEL SET', 'price': 2000, 'formula': '$1+$2', '@neededAmount': 4, '@consumedAmount': 55, '@instockAmount': 0, '@hasEnoughOfMaterial': False}}]}, {'Start': [{'v_id': 'TIRES', 'v_type': 'product', 'attributes': {'name': 'TIRES', 'price': 100, 'formula': '($1+$2)*0.8', '@neededAmount': 64, '@consumedAmount': 960, '@instockAmount': 0, '@hasEnoughOfMaterial': False}}, {'v_id': 'TRUCK RIMS', 'v_type': 'product', 'attributes': {'name': 'TRUCK RIMS', 'price': 110, 'formula': '($1+$2+$3)*0.8', '@neededAmount': 64, '@consumedAmount': 880, '@instockAmount': 0, '@hasEnoughOfMaterial': False}}]}, {'Start': [{'v_id': 'RUBBER', 'v_type': 'product', 'attributes': {'name': 'RUBBER', 'price': 10, 'formula': '', '@neededAmount': 6400, '@consumedAmount': 96000, '@instockAmount': 100, '@hasEnoughOfMaterial': False}}, {'v_id': 'ABS MODULE', 'v_type': 'product', 'attributes': {'name': 'ABS MODULE', 'price': 32, 'formu

In [ ]:
#processing and analysing the checkStocking results
checkStock_df=pd.DataFrame(checkStock)
checkStock_df=checkStock_df.drop(['@@displaySet'], axis = 1)
for i in range(len(checkStock_df)) :
  if(checkStock_df.loc[i, 'Start']==[]):
    checkStock_df=checkStock_df.drop(i)
checkStock_df_vals = checkStock_df[~checkStock_df['Start'].isnull()] 
print(checkStock_df_vals)

                                               Start
0  [{'v_id': 'TRUCK WHEEL SET', 'v_type': 'produc...
1  [{'v_id': 'TIRES', 'v_type': 'product', 'attri...
2  [{'v_id': 'RUBBER', 'v_type': 'product', 'attr...
3  [{'v_id': 'CONTROLLER', 'v_type': 'product', '...


In [ ]:
ProductNames=[]
Prices=[]
IsMaterialAvailable=[]
AmountNeeded=[]
InstockAmount=[]
ConsumedAmount=[]
for i in range(len(checkStock_df_vals)) :
  print(checkStock_df_vals.loc[i,'Start'][0]['attributes']['name'])
  print(checkStock_df_vals.loc[i,'Start'][0]['attributes']['price'])
  print(checkStock_df_vals.loc[i,'Start'][0]['attributes']['@hasEnoughOfMaterial'])
  ProductNames.append(checkStock_df_vals.loc[i,'Start'][0]['attributes']['name'])
  Prices.append(checkStock_df_vals.loc[i,'Start'][0]['attributes']['price'])
  IsMaterialAvailable.append(checkStock_df_vals.loc[i,'Start'][0]['attributes']['@hasEnoughOfMaterial'])
  AmountNeeded.append(checkStock_df_vals.loc[i,'Start'][0]['attributes']['@neededAmount'])
  InstockAmount.append(checkStock_df_vals.loc[i,'Start'][0]['attributes']['@instockAmount'])
  ConsumedAmount.append(checkStock_df_vals.loc[i,'Start'][0]['attributes']['@consumedAmount'])
processed_checkStock_data= pd.DataFrame(list(zip(ProductNames, Prices,IsMaterialAvailable,AmountNeeded,InstockAmount,ConsumedAmount)),
               columns =['ProductNames', 'Prices','IsMaterialAvailable','AmountNeeded','InstockAmount','ConsumedAmount'])
print(processed_checkStock_data)

TRUCK WHEEL SET
2000
False
TIRES
100
False
RUBBER
10
False
CONTROLLER
32
False
      ProductNames  Prices  IsMaterialAvailable  AmountNeeded  InstockAmount  \
0  TRUCK WHEEL SET    2000                False             4              0   
1            TIRES     100                False            64              0   
2           RUBBER      10                False          6400            100   
3       CONTROLLER      32                False           192            100   

   ConsumedAmount  
0              55  
1             960  
2           96000  
3            2800  


In [ ]:
#shows the raw material amount needed to produce a product
fig = px.pie(processed_checkStock_data, values="AmountNeeded", names="ProductNames",
             title='Amount Needed for Product Production',
             hover_data=['AmountNeeded'], labels={'Names':'Product Names', 'AmountNeeded':'Amount Required'})
fig.show()

In [ ]:
#shows the amount needed and consumed for the required raw materials
fig = px.scatter(processed_checkStock_data, x="AmountNeeded", y="ConsumedAmount",
	         size="Prices", color="ProductNames",
                 hover_name="ProductNames", hover_data=processed_checkStock_data.columns,log_x=True, size_max=60)
fig.show()

In [ ]:
#analysing the production capacity i.e., raw materail availability
fig = px.histogram(processed_checkStock_data, x="Prices", y="ProductNames", color="IsMaterialAvailable",
                   marginal="box", # or violin, rug
                   hover_data=processed_checkStock_data.columns)
fig.show()

**impactAnalysis**
This query analyze the effect of site down on other products and sites.

In [ ]:
#select down site name and max-depth
site_drop_down=widgets.Dropdown(
    options=site_list,
    description='Select a production site name',
    disabled=False,
)
display(site_drop_down)
max_depth_drop_down=widgets.Dropdown(
    options=max_depth_list,
    description='Select the max-depth',
    disabled=False,
)
display(max_depth_drop_down)

Dropdown(description='Select a production site name', options=('site1', 'site2', 'site3', 'site4', 'site5', 's…

Dropdown(description='Select the max-depth', options=(2, 4, 5, 8, 10), value=2)

In [ ]:
#run impactAnalysis query
site=site_drop_down.value
max_depth=max_depth_drop_down.value
params = {"affectedSites":site, "maxDepth":max_depth} #query's arguments
queryName = "impactAnalysis"
impactAnalysis = conn.runInstalledQuery(queryName, params) 
print(impactAnalysis)

[{'@@resultSet': [{'e_type': 'produce', 'from_id': 'site1', 'from_type': 'site', 'to_id': 'TRUCK WHEEL SET', 'to_type': 'product', 'directed': True, 'attributes': {}}]}]


In [ ]:
#processing and analysing the impactAnalysis results
impactAnalysis_df = pd.DataFrame(impactAnalysis[0]['@@resultSet'])
print(impactAnalysis_df)
impactAnalysis_graph = graphistry.edges(impactAnalysis_df, 'from_id', 'to_id')

    e_type from_id from_type            to_id  to_type  directed attributes
0  produce   site1      site  TRUCK WHEEL SET  product      True         {}


In [ ]:
impactAnalysis_graph.plot()

Failed memoization speedup attempt due to Pandas internal hash function failing. Continuing without memoization speedups.This is fine, but for speedups around skipping re-uploads of previously seen tables, try identifying which columns have types that Pandas cannot hash, and convert them to hashable types like strings.


**pricePrediction**
This query analyses if the price of one or more product changes, how will it effect the prices of other products. It is a iterative query.

In [ ]:
#select material name, change in material prices
material_drop_down=widgets.Dropdown(
    options=material_list,
    description='Select a material name',
    disabled=False,
)
display(material_drop_down)
price_increment_drop_down=widgets.Dropdown(
    options=price_increment_list,
    description='Select the change in material prices ',
    disabled=False,
)
display(price_increment_drop_down)

Dropdown(description='Select a material name', options=('RUBBER', 'TIRES', 'STEEL RODS', 'ALUMINUM RODS', 'SEN…

Dropdown(description='Select the change in material prices ', options=(0.2, 0.4, 0.6, 0.8, 1), value=0.2)

In [ ]:
#run pricePrediction query
material=material_drop_down.value
increment=price_increment_drop_down.value
inp=material+','+str(increment)
params = {"input":inp, "maxIteration":2,"doUpdate":False} #query's arguments
queryName = "pricePrediction"
pricePrediction = conn.runInstalledQuery(queryName, params) 
print(pricePrediction)

[{'@@depth': 1}, {'@@depth': 2}, {'@@displaySet': [{'e_type': 'reverseUsedBy', 'from_id': 'TRUCK WHEEL SET', 'from_type': 'product', 'to_id': 'TIRES', 'to_type': 'product', 'directed': True, 'attributes': {'formula_order': '1', 'useAmount': 16}}, {'e_type': 'reverseUsedBy', 'from_id': 'CAR WHEEL SET', 'from_type': 'product', 'to_id': 'CAR RIMS', 'to_type': 'product', 'directed': True, 'attributes': {'formula_order': '2', 'useAmount': 4}}, {'e_type': 'reverseUsedBy', 'from_id': 'CAR WHEEL SET', 'from_type': 'product', 'to_id': 'TIRES', 'to_type': 'product', 'directed': True, 'attributes': {'formula_order': '1', 'useAmount': 4}}, {'e_type': 'reverseUsedBy', 'from_id': 'TRUCK WHEEL SET', 'from_type': 'product', 'to_id': 'TRUCK RIMS', 'to_type': 'product', 'directed': True, 'attributes': {'formula_order': '2', 'useAmount': 16}}, {'e_type': 'reverseUsedBy', 'from_id': 'TIRES', 'from_type': 'product', 'to_id': 'RUBBER', 'to_type': 'product', 'directed': True, 'attributes': {'formula_order': 

In [ ]:
#processing and analysing the pricePrediction results
pricePrediction_df = pd.DataFrame(pricePrediction[2]['@@displaySet'])
print(pricePrediction_df )
pricePrediction_graph=graphistry.edges(pricePrediction_df , 'from_id', 'to_id')

          e_type          from_id from_type       to_id  to_type  directed  \
0  reverseUsedBy  TRUCK WHEEL SET   product       TIRES  product      True   
1  reverseUsedBy    CAR WHEEL SET   product    CAR RIMS  product      True   
2  reverseUsedBy    CAR WHEEL SET   product       TIRES  product      True   
3  reverseUsedBy  TRUCK WHEEL SET   product  TRUCK RIMS  product      True   
4  reverseUsedBy            TIRES   product      RUBBER  product      True   
5  reverseUsedBy            TIRES   product       BELTS  product      True   

                                 attributes  
0   {'formula_order': '1', 'useAmount': 16}  
1    {'formula_order': '2', 'useAmount': 4}  
2    {'formula_order': '1', 'useAmount': 4}  
3   {'formula_order': '2', 'useAmount': 16}  
4  {'formula_order': '2', 'useAmount': 100}  
5  {'formula_order': '1', 'useAmount': 200}  


In [ ]:
pricePrediction_graph.plot()

Failed memoization speedup attempt due to Pandas internal hash function failing. Continuing without memoization speedups.This is fine, but for speedups around skipping re-uploads of previously seen tables, try identifying which columns have types that Pandas cannot hash, and convert them to hashable types like strings.


In [ ]:
pricePrediction_data = pd.DataFrame(pricePrediction[3]['final'])
pricePrediction_data=pricePrediction_data['attributes']
print(pricePrediction_data)

0    {'name': 'CAR RIMS', 'price': 80, 'formula': '...
1    {'name': 'TIRES', 'price': 0.2, 'formula': '($...
2    {'name': 'TRUCK RIMS', 'price': 110, 'formula'...
3    {'name': 'RUBBER', 'price': 0.2, 'formula': ''...
4    {'name': 'BELTS', 'price': 0.2, 'formula': '$1...
Name: attributes, dtype: object


In [ ]:
Names=[]
Old_Prices=[]
New_Prices=[]
for i in range(len(pricePrediction_data)) :
  print(pricePrediction_data.loc[i]['name'])
  print(pricePrediction_data.loc[i]['price'])
  print(pricePrediction_data.loc[i]['@new_price'])
  Names.append(pricePrediction_data.loc[i]['name'])
  Old_Prices.append(pricePrediction_data.loc[i]['price'])
  New_Prices.append(pricePrediction_data.loc[i]['@new_price'])

CAR RIMS
80
0
TIRES
0.2
112.00001
TRUCK RIMS
110
0
RUBBER
0.2
0
BELTS
0.2
0.6


In [ ]:
processed_pricePrediction_data= pd.DataFrame(list(zip(Names,Old_Prices,New_Prices)),
               columns =['Names', 'Old_Prices','New_Prices'])
print(processed_pricePrediction_data)

        Names  Old_Prices  New_Prices
0    CAR RIMS        80.0     0.00000
1       TIRES         0.2   112.00001
2  TRUCK RIMS       110.0     0.00000
3      RUBBER         0.2     0.00000
4       BELTS         0.2     0.60000


In [ ]:
#shows the comparative analysis of product new price and old price
fig = px.scatter_3d(processed_pricePrediction_data, x="Names", y="Old_Prices", z="New_Prices",color="Names")
fig.show()

In [ ]:
#shows the old prices of products
fig = px.bar(processed_pricePrediction_data, x="Names", y="Old_Prices")
fig.show()

In [ ]:
#shows the comparative analysis of product new price and old price
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Old_Prices', x=processed_pricePrediction_data['Names'], y=processed_pricePrediction_data['Old_Prices']),
    go.Bar(name='New_Prices', x=processed_pricePrediction_data['Names'], y=processed_pricePrediction_data['New_Prices'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()